In [1]:
library(keras)

In [2]:
train_path <- "dogs_cats_small/train/"
test_path <- "dogs_cats_small/test/"
validation_path <- "dogs_cats_small/validation/"

In [3]:
# train generator
train_augmentor = image_data_generator(
  rescale = 1/255,
  rotation_range = 300,
  width_shift_range = 0.15,
  height_shift_range = 0.15,
  shear_range = 0.2,
  zoom_range = 0.2,
  horizontal_flip = TRUE,
  fill_mode = "nearest"
)

# test generator
test_augmentor <- image_data_generator(rescale = 1/255)  

In [4]:
# load train data
train_data <- flow_images_from_directory(
    train_path,
    train_augmentor,
    target_size = c(150, 150),
    batch_size = 20,
    class_mode = "binary")

# load test data
test_data <- test_generator <- flow_images_from_directory(
    test_path,
    test_augmentor,
    target_size = c(150, 150),
    batch_size = 20,
    class_mode = "binary")

# load validataion data
validation_data <- flow_images_from_directory(
  validation_path,
  test_augmentor,
  target_size = c(150, 150),
  batch_size = 20,
  class_mode = "binary"
)

In [5]:
train_data$image_shape

[[1]]
[1] 150

[[2]]
[1] 150

[[3]]
[1] 3

In [6]:
pre_trained_base <- application_vgg16(
  weights = "imagenet",
  include_top = FALSE,
  input_shape = c(150, 150, 3)
)

In [7]:
summary(pre_trained_base)

________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input_1 (InputLayer)                (None, 150, 150, 3)             0           
________________________________________________________________________________
block1_conv1 (Conv2D)               (None, 150, 150, 64)            1792        
________________________________________________________________________________
block1_conv2 (Conv2D)               (None, 150, 150, 64)            36928       
________________________________________________________________________________
block1_pool (MaxPooling2D)          (None, 75, 75, 64)              0           
________________________________________________________________________________
block2_conv1 (Conv2D)               (None, 75, 75, 128)             73856       
________________________________________________________________________________
block2_conv2 (Conv2D)       

In [8]:
model_with_pretrained <- keras_model_sequential() %>% 
    pre_trained_base %>% 
    layer_flatten() %>% 
    layer_dense(units = 8, activation = "relu") %>%
    layer_dense(units = 16, activation = "relu") %>%
    layer_dense(units = 1, activation = "sigmoid")

In [9]:
summary(model_with_pretrained)

________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
vgg16 (Model)                       (None, 4, 4, 512)               14714688    
________________________________________________________________________________
flatten (Flatten)                   (None, 8192)                    0           
________________________________________________________________________________
dense (Dense)                       (None, 8)                       65544       
________________________________________________________________________________
dense_1 (Dense)                     (None, 16)                      144         
________________________________________________________________________________
dense_2 (Dense)                     (None, 1)                       17          
Total params: 14,780,393
Trainable params: 14,780,393
Non-trainable params: 0
_______________________________

In [10]:
length(model_with_pretrained$trainable_weights)

[1] 32

In [11]:
freeze_weights(pre_trained_base)

In [12]:
length(model_with_pretrained$trainable_weights)

[1] 6

In [13]:
# Compile model
model_with_pretrained %>% compile(
  loss = "binary_crossentropy",
  optimizer = optimizer_rmsprop(lr = 0.0001),
  metrics = c('accuracy')
)

In [14]:
# Train model
model_with_pretrained %>% fit_generator(generator = train_data,
                                        steps_per_epoch = 20,
                                        epochs = 10,
                                        validation_data = validation_data)

In [15]:
model_with_pretrained %>% save_model_hdf5("model_with_pretrained.h5")

# cnn_model <- load_model_hdf5("cnn_model_pool.h5")

In [16]:
scores <- model_with_pretrained %>% evaluate_generator(generator = test_data,steps = 20)

In [17]:
# Output metrics
paste('Test loss:', scores[[1]])
paste('Test accuracy:', scores[[2]])

[1] "Test loss: 0.385350868999958"

[1] "Test accuracy: 0.830999970436096"

**There is more..**

In [ ]:
unfreeze_weights(pre_trained_base, from = "block5_conv1", to = "block5_conv3")

In [21]:
length(model_with_pretrained$trainable_weights)

[1] 12